## ctcsound7

The Csound API is a set of C functions and C++ classes that expose to hosts programs the functionalities of Csound.

**ctcsound7** is a python module wrapping the access to the Csound API using two Python classes: *Csound* and *CsoundPerformanceThread*. **ctcsound** uses the **ctypes** python module which is an FFI (Foreign Function Interface) allowing Python to access C shared libraries. 

In [1]:
import ctcsound7 as ct

## Csound Instance

Creating a `Csound` instance spawns a csound process.

In [2]:
cs = ct.Csound()

This object can return useful information about the underlying process

In [3]:
cs.version()

7000

In [4]:
from pprint import pprint

csoundversion = cs.version()
majorver = csoundversion // 1000
minorver = (csoundversion % 1000) // 100
patch = csoundversion % 10

print(f"Version id: {csoundversion}, version tuple: {majorver}.{minorver}.{patch}")

# On csound 6 there is an api version, this does not exist in csound 7
if csoundversion < 7000:
    print(f"API version: {cs.APIVersion()}")

# List of available opcodes, print only the beginning of the list
pprint(cs.getOpcodes()[:20])

# Opcodes with 'osc' in the name
print("\n--- 'osc' opcodes ---\n")
pprint(set(opc.name for opc in cs.getOpcodes() if "osc" in opc.name))

Version id: 7000, version tuple: 7.0.0
[OpcodeDef(name='ATSadd', outtypes='a', intypes='kkSiiopo', flags=0),
 OpcodeDef(name='ATSadd', outtypes='a', intypes='kkiiiopo', flags=0),
 OpcodeDef(name='ATSaddnz', outtypes='a', intypes='kSiop', flags=0),
 OpcodeDef(name='ATSaddnz', outtypes='a', intypes='kiiop', flags=0),
 OpcodeDef(name='ATSbufread', outtypes='', intypes='kkSiop', flags=0),
 OpcodeDef(name='ATSbufread', outtypes='', intypes='kkiiop', flags=0),
 OpcodeDef(name='ATScross', outtypes='a', intypes='kkSikkiopoo', flags=0),
 OpcodeDef(name='ATScross', outtypes='a', intypes='kkiikkiopoo', flags=0),
 OpcodeDef(name='ATSinfo', outtypes='i', intypes='Si', flags=0),
 OpcodeDef(name='ATSinfo', outtypes='i', intypes='ii', flags=0),
 OpcodeDef(name='ATSinterpread', outtypes='k', intypes='k', flags=0),
 OpcodeDef(name='ATSpartialtap', outtypes='kk', intypes='i', flags=0),
 OpcodeDef(name='ATSread', outtypes='kk', intypes='kSi', flags=0),
 OpcodeDef(name='ATSread', outtypes='kk', intypes='ki

In [5]:
del cs

		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 2.530s, CPU: 0.152s


## Performing an orchestra

The user has the option to perform an external .csd file, the text inside the .csd file or just pass csound code to the `Csound` instance to compile. 

In this case we will be performing in real-time, so we need to set the output to `dac`. 

In [ ]:
cs = ct.Csound()
cs.setOption('-odac')

Now we need to compile some actual code. Notice that settings like ksmps and 0dbfs need to be set at the first compile action. 

In [ ]:
cs.compileOrc(r'''
sr = 44100
ksmps = 64
0dbfs = 1

instr 1
  ; set default values
  ;             p4   p5  p6    p7
  pset 0, 0, 0, 0.1, 60, 0.01, 0.2
  iamp = p4
  ipitch = p5
  iattack = p6
  irelease = p7
  a0 = vco2(iamp, mtof(ipitch)) * linsegr:a(0, iattack, 1, irelease, 0)
  outch 1, a0
endin
''')

In [4]:
score = [
    (1, 0, 10, 0.1, 60),
    (1, 0.5, 10, 0.05, 62)   
]

for event in score:
    cs.scoreEvent('i', event)

To actually perform the score we need to call `.perform()`, which executes all the events scheduled until now

In [ ]:
cs.perform()

--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
ALSA output: total buffer size: 1024, period size: 256
writing 256 sample blks of 64-bit floats to dac
SECTION 1:
new alloc for instr 1:
	   T  0.501 TT  0.501 M:  0.11661
new alloc for instr 1:


If some kind of interaction between the running csound process and python is needed, it is possible to use `.performKsmps()`, which processes one cycle of samples (determined by `ksmps`) and yields control to the user.

In the example below we use this to schedule events by calculating time from python. This is just to show that any kind of process from python can be used to interact with a running csound instance

In [69]:
import random
import time

# We need this in order to be able to use keyboard interrupt. Otherwise csound
# itself sets a signal handler and python looses control 
ct.csoundInitialize(signalHandler=False, atExitHandler=False)

cs = ct.Csound()

# Realtime processing
cs.setOption('-odac')

# Disable printing whenever a new event is scheduled
cs.setOption('-m128')

# The orchestra
cs.compileOrc(r'''
sr = 44100
nchnls = 2
ksmps = 64
0dbfs = 1

instr 1
  iamp = p4
  ipitch = p5
  iattack = p6 > 0 ? p6 : 0.01
  irelease = p7 > 0 ? p7 : 0.4
  a0 = oscili(iamp, mtof(ipitch)) + oscili(iamp, mtof(ipitch+0.12))
  outall a0 * linsegr:a(0, iattack, 1, iattack*2, 0.2, irelease, 0)
endin
''')

print("\nPress I-I to interrupt\n")

# Schedule a note every 1/8 of a second. The pitch is chosen at random
# from a given scale, in this case a c-major scale
t0 = time.time()
scale = [0, 2, 4, 5, 7, 9, 11]
pitches = [60 + 12 * octave + step for octave in [0, 1] for step in scale]
while cs.performKsmps() == ct.CSOUND_SUCCESS:
    t1 = time.time()
    if t1 - t0 > 1/8.:
        pitch = random.choice(pitches)
        if pitch % 12 in (2, 4, 7):
            dur = 1.5
        else:
            dur = 0.15
        cs.scoreEvent('i', (1, 0, dur, 0.1, random.choice(pitches)))
        t0 = t1



Press I-I to interrupt



--Csound version 7.0 (double samples) Dec  4 2024
[commit: ababd1a5e09ada51e5013f24732265a4273f9f09]
libsndfile-1.2.2
graphics suppressed, ascii substituted
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 256 sample-frame blocks
ALSA output: total buffer size: 1024, period size: 256
writing 512 sample blks of 64-bit floats to dac
SECTION 1:


KeyboardInterrupt: 

The same can be achieved with csound counting time

In [70]:
sr = cs.sr()
t0 = cs.currentTimeSamples() / sr

while cs.performKsmps() == ct.CSOUND_SUCCESS:
    t1 = cs.currentTimeSamples() / sr
    if t1 - t0 > 1/8.:
        pitch = random.choice(pitches)
        if pitch % 12 in (2, 4, 7):
            dur = 1.5
        else:
            dur = 0.15
        cs.scoreEvent('i', (1, 0, dur, 0.1, random.choice(pitches)))
        t0 = t1


KeyboardInterrupt: 